Импортируем нужные для работы библиотеки

In [ ]:
! pip install spacy
! python -m spacy download ru_core_news_sm

In [2]:
import pandas as pd
import unicodedata
import re
import spacy
import json
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

# Предобработка данных

Загрузим данные

In [3]:
data = pd.read_csv('csv\\full_df.csv', sep=';')
data.head(3)

,Unnamed: 0,status,url,text,likes,reposts,comments
0,0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0
1,1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0
2,2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  11238 non-null  int64  
 1   status      11238 non-null  object 
 2   url         11238 non-null  object 
 3   text        11238 non-null  object 
 4   likes       11081 non-null  object 
 5   reposts     10908 non-null  float64
 6   comments    10902 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 614.7+ KB


In [5]:
data = data.drop(columns='Unnamed: 0', axis=1)

Отфильтруем данные по количеству символов в постах. Оставим посты только длиннее 100 символов.

In [6]:
data['text_len'] = data.loc[:, 'text'].apply(lambda x: len(x))
data_filtered_by_text_len = data.query('text_len > 100')
print(f'Количество постов с 100+ символами: {data_filtered_by_text_len.shape[0]}')
data_filtered_by_text_len.head(3)

Количество постов с 100+ символами: 8556


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Для обработки нам нужны только посты на кириллице. Отфильтруем посты содержащие кириллицу.

In [7]:
data_cyrillic = data_filtered_by_text_len[data_filtered_by_text_len['text'].apply(lambda x: re.match(r'[А-Яа-я]+', x) is not None)]

print(f'Текстов на кириллице: {data_cyrillic.shape[0]}')
data_cyrillic.head(3)

Текстов на кириллице: 1960


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Удалим дубликаты в столбце с текстами

In [8]:
data_dd = data_cyrillic.drop_duplicates('text')
print(f'Осталось {data_dd.shape[0]} строк')
data_dd.head(3)

Осталось 1862 строк


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
6,Talent Acquisition Manager | Recruitment Lead ...,https://www.linkedin.com/in/dariaivanova,С трепетом готов представить свой продукт. Реф...,8.0,0.0,0.0,841


# Подготовка текстов

In [9]:
model = spacy.load('ru_core_news_sm', disable=['ner', 'parser'])

Складываем все посты в один список

In [10]:
texts = data_dd["text"].tolist()

Выполняем лемматизацию текстов. Наберитесь терпения - это долгий процесс, который может занять около часа.

In [11]:
res = []
for doc in model.pipe(texts, disable=["tagger", "parser"]):
    res.append([token.lemma_ for token in doc])

Сохраним результат, чтобы не повторять лемматизацию заново в следующий раз.

In [12]:
with open("lemmas.json", "w") as fid:
    json.dump({"lemmas": res}, fid)

Смотрим на результат - случайным образом сэмплируем несколько примеров

In [13]:
", ".join(res[np.random.randint(len(res))])

'наш, команда, успешно, работать, не, только, с, финансовый, сектор, и, ритейлом, ,, но, и, с, производственными, компания, ., нужный, консультация, по, автоматизация, бизнес, -, процессов, в, ваш, компания, ,, выбор, вендор, или, решение, ?, написать, мне, в, личный, сообщения.#crmguru, #, feedback'

In [14]:
", ".join(res[np.random.randint(len(res))])

'друг, ,, в, крупный, онлайн, -, гипермаркет, детский, товар, искать, web, -, аналитика, с, хороший, опыт, в, google, analytics, и, владение, sql, на, уровень, написание, запрос, ., по, функция, это, участие, в, развёртывание, система, google, analytics, premium, и, аналитический, поддержка, канал, продажа, ., предлагать, соц, ., пакет, (, дмс, ,, фитнес, ,, корпоративный, изучение, язык, ), ,, комфортабельный, офис, ,, вилка, заработный, плата, ,, которая, формироваться, с, учёт, ожидание, кандидат, ., связаться, со, мной, можно, личный, сообщение, либо, написать, на, почта, klevanets@ackvius.ru, ., также, быть, благодарный, за, рекомендация, .'

Удалим знаки препинания, цифры и слова, содержащие цифры. Для это соберём регулярное выражение, которое подходит для любого слова, состоящего из букв кирилицы.

In [15]:
word_pattern = re.compile("^[а-я]*$")

def remove_symbols(doc):
    return [token for token in doc if word_pattern.match(token)]

corpus = list(map(remove_symbols, res))

In [16]:
", ".join(corpus[np.random.randint(len(corpus))])

'проверь, свой, в, один, клик, это, такой, и, как, его, повысить, это, внутренний, показатель, который, оценивать, насколько, активно, вы, использовать, свой, личный, профиль, для, нетворкинга, и, выстраивание, профессиональный, бренд, в, сеть, если, у, вас, высокий, показатель, выше, профиль, лучше, показываться, в, поиск, пост, получать, больше, просмотр, и, в, целое, профиль, выглядеть, более, доверительный, для, индекс, состоять, из, четыре, шкала, каждый, из, которых, иметь, вес, балл, максимальный, показатель, это, балл, это, первый, шкала, она, напрямую, связать, с, качество, ваш, профиль, а, также, с, тем, как, часто, вы, публиковать, релевантный, и, ценный, пост, это, второй, шкала, она, напрямую, связать, с, ваш, способность, находить, правильный, человек, за, минимальный, время, использовать, встроенный, поисковый, движок, на, это, третий, шкала, она, напрямую, связать, с, ваш, способность, заинтересовывать, других, человек, посредством, собственный, контент, начинать, с, он

# Векторизация текстов

Теперь необходимо перевести тексты новостей и слова, из которых они состоят, в числовое представление, т.е. провести векторизацию. Для этого можно использовать подход "мешок слов" (Bag of Words) или Tf-iDf. Второй вариант нравится мне больше.

Чтобы уменьшить размерность получаемой матрицы, добавим исключение стоп-слов.

In [17]:
stop_words = stopwords.words('russian')
print(stop_words)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [18]:
tfidf = TfidfVectorizer(stop_words=stop_words, min_df=10, max_df=0.9)

Т.е. TfIdfVectorizer принимает на вход список предложений, поэтому необходимо объединить списки слов, составляющих корпус, в предложения.

In [19]:
docs = [" ".join(tokens) for tokens in corpus]
docs[:3]

['всем свой первый статья на про история и технология разработка игра и как мы создать игра с элемент машинный обучение и выйти в ноль за месяцапри внимательный прочтение вы даже смочь запустить первый версия игра',
 'если вам интересный позалипать в слово я запустить понравиться брат шникам',
 'с трепет готовый представить свой продукт рефни первый сервис реферального рекрутинг в ит на нем каждый мочь посоветовать знакомого на открытый позиция в компания и получить за рекомендация денежный вознаграждение идея создание внешний реферального сервис зародиться у меня давно в период пандемия я посоветовать пара человек на хороший должность в разный компания их взять все счастливый тогда я и подумать пора создание команда подготовка тз изучение самых разный нюанс о которых я ранний не знать и самое волнительный поиск инвестор все получиться и весь это теперь позади а впереди только постоянный развитие продукт посмотреть что у нас выйти быть благодарный если вы поддержать меня и мой команда 

In [20]:
x = tfidf.fit_transform(docs)

In [21]:
x.shape

(1862, 1635)

Теперь составим словарь {id_токена: токен} - он пригодится нам позднее.

In [23]:
id2word = {i: token for i, token in enumerate(tfidf.get_feature_names_out())}

# LDA

Теперь можем запустить алгоритм LDA. Зададим количество тем равное 20.

In [24]:
N_TOPICS = 20

In [25]:
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=765)

In [26]:
topics = lda.fit_transform(x)

In [27]:
topics.shape

(1862, 20)

Получили массив размера 1862x20. Строки здесь - отдельные тексты, а колонки - 20 выделенных тем. В каждой ячейке стоит вероятность того, что данный текст относится к данной теме. Это то самое распределение тем над документами.

## Ключевые слова

Теперь извлечём ключевые слова для каждой из 20 тем. В свойсте components _ лежит матрица соответствия слов темам. Пройдёмся по каждой из 20 выделенных тем и для каждой возьмём ТОП-20 слов.

По очереди пройдёмся по каждой теме (первое измерение массиа) и выберем ТОП-20 слов, наиболее характерных для этой темы. Потом по индексу слова обратимся к словарю и узнаем, что это за слово.

In [28]:
for i in range(N_TOPICS):
    c = lda.components_[i, :]
    topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-20:]]
    print("Тема",i+1, ":")
    print(", ".join(topic_tokens))
    print("\n ")

Тема 1 :
проверка, зависимость, кстати, мочь, обсудить, основа, очередной, визуализация, устроить, рекламный, распознавание, тренировка, ведущий, мировой, последний, который, собес, теория, посещать, ваш

 
Тема 2 :
настоящий, обсуждение, выход, запустить, душа, эффект, понравиться, интересный, услуга, сегодня, форум, фильм, программный, привлечь, документ, размер, потеря, доклад, влияние, нанимать

 
Тема 3 :
новый, инстаграм, карточка, гость, яндекс, немного, игра, инвестиция, программист, ведущий, машинный, проектноеуправление, обучение, айти, системнаяаналитика, обучениеснуля, погнать, интеллект, подкаст, искусственный

 
Тема 4 :
просто, прочитать, таки, лекция, пока, именно, хороший, предприятие, предыдущий, данные, расход, день, подробность, оценить, просьба, сократить, кнопка, яндекс, вода, добрый

 
Тема 5 :
эмоция, взгляд, множество, запись, заявка, вчера, стратегия, давно, контент, задавать, понимание, отношение, создать, психология, код, конец, выбор, внутренний, будущее, л

По полученным ключевым словам можно составить представление о содержании той или иной темы. Хотя иногда сочетания слов довольно странные )

## Типичные статьи

Теперь для каждой темы возьмём по одному документу, который хорошо соответсвует этой теме. Для этого заглянем в массив topics.

In [29]:
for i in range(N_TOPICS):
    doc_id = np.argmax(topics[:, i])
    print("Тема ", i)
    print(data_dd.iloc[doc_id]["text"])
    print("\n")

Тема  0
Вашему вниманию успешный кейс из Казахстана.Обеспечение работоспособности и безопасности производства при снижении расхода масла на 30%.


Тема  1
За последние 1.5 года армянские стартапы привлекли суммарно более миллиарда долларов в 26 сделках. В то время как, сектор ИТ в Армении в 2022 вырос на 30%. #InvestinArmenia


Тема  2
Почти «догрыз» специализацию по машинному обучению и анализу данных от Яндекса на coursera. 5/6 - done. Так что, скоро уже прекращу постить пачками сертификаты и займусь делом наконец :)


Тема  3
Когда сейлз просит примерно оценить приложение - как убер, только для сантехников с элементами инстаграма, возможно потом добавится функционал тиндера, но это не точно. Просьба побыстрее, созвон с заказчиком через 10 мин.


Тема  4
В Израиле создали летающих роботов для сбора фруктовКаждый дрон оснащен мягким захватом. Дроны способны анализировать и проверять, где находится фрукт и какого он качества.


Тема  5
Друзі, звертаюся до спільноти! В мене є деякий віл